In [1]:
# Install required packages in the current Jupyter kernel
# Uncomment the following lines if you need to install these libraries
# If you run into permission issues, try with the --user option
#import sys
#!pip install -q rdflib networkx matplotlib
#!{sys.executable} -m pip install rdflib networkx matplotlib pandas stringdb --user


In [9]:
%load_ext autoreload
%autoreload 2
import sampler
import grabhogs_sparql
import map2string_fast
import addfrombloom
import rdflib
import SPARQLWrapper
import colour
import itertools
import networkx as nx
import glob
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
datapath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/402676.protein.links.rdf.v11.5.txt.gz'
# RDF graph loading
rg = sampler.load_graph(datapath)

datapath2 = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.info.rdf.v11.5/402676.protein.info.rdf.v11.5.txt.gz'
# RDF graph loading
rg_info  = sampler.load_graph(datapath2)

rdflib Graph loaded successfully with 2620310 triples
rdflib Graph loaded successfully with 14559 triples


In [11]:
links = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/*.protein.links.rdf.v11.5.txt.gz'
linkfiles = glob.glob(links)
linkfiles = { l:{ 'links':l , 'info':l.replace('protein.links' , 'protein.info' ) } for l in linkfiles}
print(len(linkfiles ))

1831


In [12]:
subjs = rg.subjects( unique = True)
seed = next(subjs)
print(seed)

https://string-db.org/network/402676.B6JY50


In [13]:
subg = sampler.sample( rg = rg , seed = seed,  layer_limit= 2 , sample_run = 20 )
print(set([p for p in subg.predicates()]))
print("rdflib Graph sampled successfully with {} triples".format(len(subg)))

176 699
{rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/any-confidence')}
rdflib Graph sampled successfully with 544 triples


In [14]:
subg = sampler.add_xrefs( rg_info , subg )
print("rdflib Graph annotated successfully with {} triples".format(len(subg)))
print(set([p for p in subg.predicates()]))
cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot")
i = 0 
for s,p,o in subg.triples((None, cross_ref, None)):
    print(s,p,o)
    i+= 1
    if i > 10:
        break

92
92
rdflib Graph annotated successfully with 636 triples
{rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/any-confidence'), rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot'), rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff')}
https://string-db.org/network/402676.B6K7A4 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6K7A4
https://string-db.org/network/402676.B6JX35 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JX35
https://string-db.org/network/402676.B6JVM6 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JVM6
https://string-db.org/network/402676.B6K6W0 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6K6W0
https://string-db.org/network/402676.B6JY50 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JY50
https://string-db.o

In [15]:
#proteins_by_species , results_subj_para , results_subj_ortho  = grabhogs_sparql.grab_hogs( subg , cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot") )

In [16]:
#print(results_subj_ortho)

In [17]:
orthograph =  grabhogs_sparql.grab_hogs_graph( subg , cross_ref , sparql_endpoint= None
            , USE_CASE = 1 , verbose = True , cross_ref_prop = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot"))

92


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Total time for 184 SPARQL queries: 83.47040796279907 seconds (multiple batch calls in: 0 cases)
Num errors: 1


In [18]:
from rdflib import Graph, URIRef
print("rdflib Graph annotated successfully with {} triples".format(len(orthograph)))
#get all species
taxa = [ 'protein1_uniprot_taxon_orth' , 'protein1_uniprot_taxon_para']
species = set( [ o for t in taxa  for s,p,o in orthograph.triples((None, URIRef(t), None))  ])


rdflib Graph annotated successfully with 1306257 triples


In [19]:
#get all the results for that species
taxa = [ 'orth_protein_uniprot_taxon_orth' , 'para_protein_uniprot_taxon_para']
#get all proteins for each species
prots_by_species = { spec: set([s  for s,p,o in orthograph.triples((None, URIRef(t), spec ))]) for t in taxa for spec in species  }
prots_by_species = { spec:prots_by_species[spec] for spec in prots_by_species if len(prots_by_species[spec])   }
prots_by_species = { spec:[ p.replace('http://purl.uniprot.org/uniprot/' , '' ) for p in prots_by_species[spec] ] for spec in prots_by_species }


In [20]:
server = 'dna076'
ortho_xrefgraph = map2string_fast.mapall(prots_by_species , serverurl= "http://"+server+":3030/string_fuseki/sparql" , retgraph = True)
print(len(ortho_xrefgraph))

29059


In [21]:
orthograph += ortho_xrefgraph
subg += orthograph
#we have interactions for one species and ortho info to all others

In [22]:
filters = addfrombloom.load_filters()

In [ ]:
#lets add the interactions for all using the bloom filters
print( len(orthograph ) , set([p for p in orthograph.predicates() ]) )
#get string ids by species
pred = rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot')
interactions = []
for spec in prots_by_species:
    stringids = [ s for prot in prots_by_species[spec] for s,p,o in orthograph.triples((None, pred , URIRef('http://purl.uniprot.org/uniprot/'+prot) )) ]
    stringids = [ s.replace('https://string-db.org/network/' , '' ) for s in stringids ]
    if len(stringids ) > 2 :
        interactions += addfrombloom.check_allvall( objects = stringids , urlstring = 'https://string-db.org/network/' , filters = filters )
        print(spec,stringids[0:10], len(interactions))
[subg.add(t) for t in interactions]

1335316 {rdflib.term.URIRef('protein1_uniprot_para_protein_uniprot'), rdflib.term.URIRef('taxon_taxon_para'), rdflib.term.URIRef('taxon_para_protein_uniprot'), rdflib.term.URIRef('protein1_orth_protein'), rdflib.term.URIRef('protein1_taxon'), rdflib.term.URIRef('taxon_orth_protein'), rdflib.term.URIRef('para_protein_taxon_para'), rdflib.term.URIRef('protein1_uniprot_orth_protein_uniprot'), rdflib.term.URIRef('taxon_taxon_orth'), rdflib.term.URIRef('orth_protein_uniprot_taxon_orth'), rdflib.term.URIRef('protein1_taxon_para'), rdflib.term.URIRef('protein1_para_protein_uniprot'), rdflib.term.URIRef('orth_protein_taxon_orth'), rdflib.term.URIRef('para_protein_uniprot_taxon_para'), rdflib.term.URIRef('protein1_uniprot_para_protein'), rdflib.term.URIRef('protein1_taxon_orth'), rdflib.term.URIRef('protein1_uniprot_taxon_para'), rdflib.term.URIRef('protein1_orth_protein_uniprot'), rdflib.term.URIRef('taxon_orth_protein_uniprot'), rdflib.term.URIRef('protein1_uniprot_orth_protein'), rdflib.term

In [ ]:
#halelujah we have a graph with everything in it
#serialize to turtle format
v = subg.serialize(format="ttl")
with open('testgraph.ttl', 'w') as graphout:
    graphout.write(v)

In [ ]:
readg = rdflib.Graph()
readg.parse('testgraph.ttl')
print(len(readg))
#we can save the subgraphs in rdf...

In [8]:
# create a new heterodata object
from rdflib import URIRef
from torch_geometric.data import HeteroData , Data
import torch_geometric.transforms as T
import torch_geometric.utils 
from rdflib import RDF
import numpy as np
import scipy.sparse
import torch

def sparse2pairs(sparsemat, matrows = None):
    '''
    This functino takes a sparse matrix and returns a list of pairs of the non zero entries
    args:
        sparsemat: a sparse matrix
        matrows: a list of the matrix rows to keep
    Returns:    
        a list of pairs of the non zero entries
    '''
    if matrows :
        sparsemat = sparsemat[matrows,:]
        sparsemat = sparsemat[:,matrows]
    sparsemat = scipy.sparse.find(sparsemat)
    return np.vstack([sparsemat[0],sparsemat[1]])

def test_heterograph(rdf_graph , verbose = True):
    data = HeteroData()
    # assign edge types from the predicate
    edge_types = set([p for s,p,o in rdf_graph])
    edge_types = {e:i for i,e in enumerate(edge_types)}
    uris = {}
    subtypes = {}
    for sub, pred, obj in rdf_graph:
        for o in (sub,obj):
            if isinstance(o, URIRef):
                uri_type = rdf_graph.qname(o).split(":")[0]  # get the namespace prefix
                if uri_type not in uris:
                    uris[uri_type] = []
                    subtypes[uri_type] = set([])
                uris[uri_type].append(o)
                subtypes[uri_type].add( ''.join(o.n3().split('/')[0:-1] )  )
    if verbose == True:
        for t in subtypes:
            #compile the x data matrix for each subtype
            subtype_dict = { ty:i for i,ty in enumerate(subtypes[t])}
            indices = [ subtype_dict[''.join(o.n3().split('/')[0:-1])] for o in uris[t] ]
            x= np.zeros( (len(uris[t]), len(subtype_dict)))
            x[:,indices] = 1
            #for now this is 1hot for subtype
            #add some more descriptive data here if you have it
            data[t].x = torch.tensor(x , dtype=torch.float )
    node_index_by_type = { uritype : { n:i for i,n in enumerate( set(uris[uritype]) ) } for uritype in uris }
    #todo add the diff types of uris within each namespace as 1hot encoded for x attribute of each node
    for t1,t2 in itertools.product(node_index_by_type,node_index_by_type):
            rows = node_index_by_type[t1]
            columns = node_index_by_type[t2]
            if verbose == True:
                print(t1,len(rows),t2, len(columns)) 
            for edge_type in edge_types:
                # create a dictionary of nodes
                triples =  [ (s,p,o) for s,p,o in rdf_graph.triples((None, edge_type, None))  if ( s in rows and o in columns )  ]
                if len(triples)>0:
                    adj = scipy.sparse.lil_matrix((len(rows), len(columns)))
                    for s,p,o in triples:    
                        adj[rows[s], columns[o]] = 1
                    if verbose == True:
                        print(adj.sum())
                        pass
                    #between subgraphs
                    data[ t1 , p.n3() , t2 ].edge_index = torch.tensor( sparse2pairs(adj) ,  dtype=torch.long )
                    if t1 == t2:
                        #within a subgraph of the same namespace
                        data[ t1 , p.n3() , t2 ].edge_index  = torch_geometric.utils.add_self_loops(data[ t1 , p.n3() , t2 ].edge_index )
    print(data)
    data = T.ToUndirected()(data)
    return data
data = test_heterograph(readg)
print(data)

ns3 31567 ns3 31567
3029.0
29643.0
ns3 31567 ns4 38582
36536.0
3275.0
37881.0
3384.0
36.0
ns3 31567 ns5 2450
36.0
2886.0
28647.0
24844.0
1696.0
ns3 31567 ns6 42
ns4 38582 ns3 31567
2886.0
28647.0
ns4 38582 ns4 38582
3229.0
35319.0
ns4 38582 ns5 2450
24162.0
3243.0
37198.0
1642.0
35.0
ns4 38582 ns6 42
ns5 2450 ns3 31567
28647.0
2886.0
ns5 2450 ns4 38582
3229.0
35319.0
ns5 2450 ns5 2450
535.0
2448.0
ns5 2450 ns6 42
ns6 42 ns3 31567
ns6 42 ns4 38582
ns6 42 ns5 2450
ns6 42 ns6 42
120.0
120.0
120.0
120.0
HeteroData(
  ns3={ x=[267631, 1] },
  ns4={ x=[294569, 1] },
  ns5={ x=[200436, 1] },
  ns6={ x=[1010, 1] },
  (ns3, <file:///work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/RDF_Graphnet/rdf2network/protein1_para_protein>, ns3)={ edge_index=[2] },
  (ns3, <file:///work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/RDF_Graphnet/rdf2network/protein1_orth_protein>, ns3)={ edge_index=[2] },
  (ns3, <file:///work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/RDF_Graphnet/rdf2network/protein1_orth_pro

AttributeError: 'tuple' object has no attribute 'size'

In [ ]:
#get a model
import buildmodel
model = buildmodel.HeteroGCN_linkpred()

In [ ]:
#sample edges
# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 70% of edges for message passing,
# and 30% of edges for supervision.
# We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.
# We can leverage the `RandomLinkSplit()` transform for this from PyG:
#HERE the only EDGE TYPE you would use for split would be ("protein", "interacts", "protein")
# although the graph would also contain ("protein", "isOrtho", "protein")
# maybe also species as a link?? or should it be a node feature

transform = T.RandomLinkSplit(
    num_val=0.1,  # percentage of VALIDATION
    num_test=0.1,  # percentage of TESTING
    disjoint_train_ratio=0.3,  # this somehow has to do with the 30% we need for supervision
    neg_sampling_ratio=2,  # TODO
    add_negative_train_samples=True,  # TODO
    edge_types=("user", "rates", "movie"),
    rev_edge_types=("movie", "rev_rates", "user"), 
)

train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)

print(train_data["user", "rates", "movie"].edge_label_index.size(1))
print(val_data["user", "rates", "movie"].edge_label_index.size(1))
assert train_data["user", "rates", "movie"].num_edges == 56469 
#assert train_data["user", "rates", "movie"].edge_label_index.size(1) == 24201
assert train_data["movie", "rev_rates", "user"].num_edges == 56469
# No negative edges added:
#assert train_data["user", "rates", "movie"].edge_label.min() == 1
#assert train_data["user", "rates", "movie"].edge_label.max() == 1

assert val_data["user", "rates", "movie"].num_edges == 80670
assert val_data["user", "rates", "movie"].edge_label_index.size(1) == 30249
assert val_data["movie", "rev_rates", "user"].num_edges == 80670
# Negative edges with ratio 2:1:
assert val_data["user", "rates", "movie"].edge_label.long().bincount().tolist() == [20166, 10083]



# In the first hop, we sample at most 20 neighbors.
# In the second hop, we sample at most 10 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
## HERE IS where you would basically train with both types of links, but then for testing you would only want to generate INTERACT
edge_label_index = train_data["user", "rates", "movie"].edge_label_index
edge_label = train_data["user", "rates", "movie"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[20, 10],  # TODO
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    #edge_label=edge_label,
    neg_sampling_ratio=2,  # TODO
    batch_size=128,
    shuffle=True
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "movie"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "rates", "movie"].edge_label.min() == 0
assert sampled_data["user", "rates", "movie"].edge_label.max() == 1






In [ ]:
#train


In [ ]:
#pred 